In [2]:
import numpy as np
import pandas as pd
from pprint import pprint
data = pd.read_csv('titanic.csv')

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
def checkAdult(age):
    if age>=18:
        return "Adult"
    else:
        return "Child"

data["Adult/Child"]=data["Age"].apply(checkAdult)
data.head()

,PassengerId,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Adult/Child
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Adult
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Adult
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Adult


In [5]:
data.groupby(['Gender'])['Survived'].sum()

Gender
female    233
male      109
Name: Survived, dtype: int64

In [6]:
trainingData=data[["Pclass","Adult/Child","Gender","Survived"]]
trainingData.head()

,Pclass,Adult/Child,Gender,Survived
0,3,Adult,male,0
1,1,Adult,female,1
2,3,Adult,female,1
3,1,Adult,female,1
4,3,Adult,male,0


In [7]:
def catToNum(series):
    series = series.astype('category')
    return series.cat.codes

catData=trainingData[["Pclass","Adult/Child","Gender","Survived"]].apply(catToNum)
trainingData[["Pclass","Adult/Child","Gender","Survived"]]=catData
trainingData.head()

<ipython-input-7-98ad1838faf1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingData[["Pclass","Adult/Child","Gender","Survived"]]=catData


,Pclass,Adult/Child,Gender,Survived
0,2,0,1,0
1,0,0,0,1
2,2,0,0,1
3,0,0,0,1
4,2,0,1,0


In [8]:
len(trainingData)

891

In [9]:
trainingData = trainingData.dropna()
len(trainingData)

891

In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(trainingData, test_size = 0.2)

In [11]:
train.head()

,Pclass,Adult/Child,Gender,Survived
274,2,1,0,1
249,1,0,1,0
194,0,0,0,1
551,1,0,1,0
587,0,0,1,1


In [12]:
test.head()

,Pclass,Adult/Child,Gender,Survived
780,2,1,0,1
816,2,0,0,0
667,2,1,1,0
119,2,1,0,0
420,2,1,1,0


In [13]:
len(train)

712

In [14]:
len(test)

179

In [15]:
def cal_entropy(col):
    # first convert the column to numpy array and then find the unique elements and their frequency
    col = np.array(col)
    elements,freq = np.unique(col,return_counts = True)
    # print(elements,freq)
    entropy = np.sum([(-freq[i]/np.sum(freq))*np.log2(freq[i]/np.sum(freq)) for i in range(len(elements))])
    return entropy


In [16]:
entropy = cal_entropy(train["Survived"])
print(entropy)

0.971927952687147


In [17]:
def InfoGain(data,split_attribute_name,target_name="Survived"):
    """
    1. data = The dataset for whose feature the IG should be calculated
    2. split_attribute_name = the name of the feature for which the information gain should be calculated
    3. target_name = the name of the target feature. The default for this example is "class"
    """
    #Calculate the entropy of the total dataset

    parent_entropy = cal_entropy(data[target_name])
    print(f"Total dataset/Parent Entropy: {parent_entropy}")

    #Calculate the values and the corresponding counts for the split attribute
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    print(f"Split attribute Name: {split_attribute_name}")
    print(f"Split attribute values: {vals}")
    print(f"Split attribute values counts: {counts}")

    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*cal_entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    print(f"Weighted Entropy: {Weighted_Entropy}")

    #Calculate the information gain
    print(f"Calculating Information Gain using: {parent_entropy} - {Weighted_Entropy}")
    Information_Gain = parent_entropy - Weighted_Entropy
    print(f"Information Gain: {Information_Gain}")
    return Information_Gain


In [18]:
#Test the function
InfoGain(data=train,split_attribute_name="Pclass",target_name="Survived")

Total dataset/Parent Entropy: 0.971927952687147
Split attribute Name: Pclass
Split attribute values: [0 1 2]
Split attribute values counts: [175 154 383]
Weighted Entropy: 0.8984838618750007
Calculating Information Gain using: 0.971927952687147 - 0.8984838618750007
Information Gain: 0.07344409081214631


0.07344409081214631

In [19]:
def ID3(data, features, target_attribute_name):
    # Create a root node r
    r = None

    # Set r to the mode target feature value in D
    r = data[target_attribute_name].mode()[0]
    # If all target feature values are the same, return r
    if len(np.unique(data[target_attribute_name])) <= 1:
        r = np.unique(data[target_attribute_name])[0]
        return r

    # If Feature_Attributes is empty, return r
    if len(features) == 0:
        return r

    # Att = Attribute from Feature_Attributes with the largest information gain value
    att = None
    val = [InfoGain(data, feature, target_attribute_name) for feature in features]
    largest_igv_index = np.argmax(val)
    att = features[largest_igv_index]

    # r = Att
    r = att

    # Create a new node below r for each value in Att
    tree = {att: {}}

    # For values in Att:
    for value in np.unique(data[att]):
        # Add a new node below r where node_values = (Att == value)
        node_values = (data[att] == value)
        sub_d_values = data.where(node_values).dropna()

        # If Sub_D_values is empty, add a leaf node with the mode target value in D
        if len(sub_d_values) == 0:
            tree[att][value] = data[target_attribute_name].mode()[0]
        else:
            # Else, add Sub_Tree with ID3(Sub_D_values, Feature_Attributes = Feature_Attributes without Att, Target_Attributes)
            sub_tree[att][value] = ID3(sub_d_values, [x for x in features if x != att], target_attribute_name)
            tree[r][value] = sub_tree
    return tree


In [22]:
#tree = ID3(train,train,train.columns[:-1])
#pprint(tree)